<a href="https://colab.research.google.com/github/Sudheendra-RD/CNN/blob/main/CNN_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!pip install tensorflow

In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [4]:
image_size = [224,224]

train_path = '/content/drive/MyDrive/Colab Notebooks/CNN/train'
test_path = '/content/drive/MyDrive/Colab Notebooks/CNN/test'

In [5]:
# Import ResNet50 and give input size as 224,224,3. this is the standard image size for most of the pre-trained models
# "Imagenet" is a global challenge where new CNN architectures are tested with 1000s of data
# so they have trained various pre-built models with certain weights. hence we use'imagenet' weights
# Hence, pre-trained models are much more effective in CNN
# include_top= whether to inclue the input and output layer or not. if, false, we need to add a custom input and output layer.

resnet = ResNet50(input_shape=image_size + [3], weights='imagenet', include_top=False)

94781440/94765736 [==============================] - 0s 0us/step


In [6]:
# We should not train the ResNet50 model or any pre-trained CNN models as the weights get altered.
for layer in resnet.layers:
  layer.trainable = False

In [7]:
# Glob is a function used to get the number of output classes
# Here, we have Glaucoma or Non-Glaucoma. Hence, it is 2.
# While creating a folder of images, we need to seperate the images based on classes/outputs
folders = glob('/content/drive/MyDrive/Colab Notebooks/CNN/train/*')

In [8]:
# This shows we have 2 output
len(folders)

2

In [9]:
# Now we need to flatten all the ResNet50 layers to make it a 2D array
x = Flatten()(resnet.output)

In [10]:
# We need to add an output layer with Sigmoid AF since there are 2 output

prediction = Dense(len(folders), activation='sigmoid')(x)

# We create a model by specifying the input layers
model = Model(inputs=resnet.input, outputs=prediction)

In [11]:
# Review the model architecture

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [12]:
# Specifying the loss function and optimizer
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [13]:
# Data Augmentation with ImageData Generator

# We need to rescale the images from 0-255 to 0-1

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

# In the test data, we only need to rescale.
# we should not use data augmentation

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/CNN/train',
                                                 target_size = (224,224),
                                                 batch_size = 4,
                                                 class_mode = 'categorical')

Found 80 images belonging to 2 classes.


In [15]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/CNN/test',
                                            target_size = (224,224),
                                            batch_size = 4,
                                            class_mode = 'categorical')

Found 10 images belonging to 2 classes.


In [16]:
# Model fitting
fit = model.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 10,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/10
20/20 [==============================] - 53s 2s/step - loss: 1.9006 - accuracy: 0.5375 - val_loss: 0.7483 - val_accuracy: 0.5000
Epoch 2/10
20/20 [==============================] - 16s 790ms/step - loss: 0.9504 - accuracy: 0.6250 - val_loss: 1.4239 - val_accuracy: 0.5000
Epoch 3/10
20/20 [==============================] - 16s 789ms/step - loss: 0.7892 - accuracy: 0.6875 - val_loss: 0.4000 - val_accuracy: 0.7000
Epoch 4/10
20/20 [==============================] - 16s 791ms/step - loss: 0.4197 - accuracy: 0.8000 - val_loss: 0.1671 - val_accuracy: 0.9000
Epoch 5/10
20/20 [==============================] - 16s 793ms/step - loss: 0.2331 - accuracy: 0.9375 - val_loss: 0.1312 - val_accuracy: 1.0000
Epoch 6/10
20/20 [==============================] - 16s 793ms/step - loss: 0.4232 - accuracy: 0.7750 - val_loss: 0.2618 - val_accuracy: 0.8000
Epoch 7/10
20/20 [==============================] - 16s 790ms/step - loss: 0.4962 - accuracy: 0.8250 - val_loss: 0.0879 - val_accuracy: 1.0000
Ep

In [47]:
# Since this is a huge model, it is better to save the weights of each layer
# In future, we can extract the weights from this file with less time.
# We save it as .h5 file

from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/Colab Notebooks/CNN/CNN_resnet50.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [18]:
predict = model.predict(test_set)

In [41]:
# Lets check the accuracy of the model by giving an image to predict

img = image.load_img('/content/drive/MyDrive/Colab Notebooks/CNN/uploads/nonGlaucoma.jpeg', target_size=(224,224))

In [42]:
x = image.img_to_array(img)

In [43]:
x = x/255

In [44]:
x.shape

(224, 224, 3)

In [45]:
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [46]:
model.predict(img_data)

array([[9.9752444e-01, 6.5276027e-04]], dtype=float32)